In [1]:
import re
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/katiegarrett/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
top_2021 = pd.read_csv('../data/top_2021.csv')
top_2021.head()

,Unnamed: 0,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,...,created,gilded,author_is_blocked,is_created_from_ads_ui,author_flair_background_color,author_flair_template_id,author_flair_text_color,removed_by_category,author_cakeday,date
0,0,"[{'award_sub_type': 'GLOBAL', 'award_type': 'g...",True,bjrob,NaN,[],NaN,text,t2_1q50a99d,False,...,1.614811e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-03 17:43:17+00:00
1,1,"[{'award_sub_type': 'GLOBAL', 'award_type': 'g...",False,Go9752788Stay37726,NaN,[],NaN,text,t2_5h8qz4ia,False,...,1.614221e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-02-24 21:45:44+00:00
2,2,"[{'award_sub_type': 'GLOBAL', 'award_type': 'g...",True,manoffewwords,NaN,[],NaN,text,t2_38lo5,False,...,1.612395e+09,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-02-03 18:35:05+00:00
3,3,"[{'award_sub_type': 'GLOBAL', 'award_type': 'g...",True,SulkingDeath,NaN,[],NaN,text,t2_3z0ecem5,False,...,1.638933e+09,NaN,False,False,NaN,NaN,NaN,NaN,NaN,2021-12-07 22:08:11+00:00
4,4,"[{'award_sub_type': 'GLOBAL', 'award_type': 'g...",True,textmasterj,NaN,[],NaN,text,t2_4hvmd,False,...,1.614273e+09,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-02-25 12:15:16+00:00


In [3]:
top_2021["combined"] = top_2021["title"].astype(str) + top_2021["selftext"].astype(str)

In [4]:
top_2021["row_id"] = top_2021.index + 1 
print (top_2021.head(10))

   Unnamed: 0                                      all_awardings  \
0           0  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   
1           1  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   
2           2  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   
3           3  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   
4           4  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   
5           5  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   
6           6  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   
7           7  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   
8           8  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   
9           9  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   

   allow_live_comments               author  author_flair_css_class  \
0                 True                bjrob                     NaN   
1                False   Go9752788Stay37726                     NaN   
2                 True        manoffew

In [5]:
top_2021_subset = top_2021[['row_id', 'combined']].copy()
#data clean-up
#remove all non-alphabet characters
top_2021_subset['combined'] = top_2021_subset['combined'].str.replace("[^a-zA-Z#]", " ")
#convert to lower-case
top_2021_subset['combined'] = top_2021_subset['combined'].str.casefold()
print (top_2021_subset.head(10))

   row_id                                           combined
0       1  cvs is vaccinating teachers regardless of stat...
1       2  i was observed today  online learning  and i h...
2       3  has anyone else been unironically shown sir ke...
3       4  i literally just stopped teaching in the middl...
4       5  why do you want to send your kids back to us  ...
5       6  to everyone using student mental health and ac...
6       7  gay teacher here  admin told me to silence on ...
7       8  where i live  lawyers were allowed to get vacc...
8       9  tifu  inadvertently taught students how to che...
9      10  stop celebrating exploitationthe endless stori...


/var/folders/4_/948td82x7jlf4pz0c4dtqn_00000gn/T/ipykernel_31485/3027894270.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  top_2021_subset['combined'] = top_2021_subset['combined'].str.replace("[^a-zA-Z#]", " ")


In [6]:
df1=pd.DataFrame()
df1['row_id']=['99999999999']
df1['sentiment_type']='NA999NA'
df1['sentiment_score']=0

In [7]:
print('Processing sentiment analysis...')
sid = SentimentIntensityAnalyzer()
t_df = df1
for index, row in top_2021_subset.iterrows():
    scores = sid.polarity_scores(row[1])
    for key, value in scores.items():
        temp = [key,value,row[0]]
        df1['row_id']=row[0]
        df1['sentiment_type']=key
        df1['sentiment_score']=value
        t_df=t_df.append(df1)
#remove dummy row with row_id = 99999999999
t_df_cleaned = t_df[t_df.row_id != '99999999999']
#remove duplicates if any exist
t_df_cleaned = t_df_cleaned.drop_duplicates()
# only keep rows where sentiment_type = compound
t_df_cleaned = t_df[t_df.sentiment_type == 'compound']
print(t_df_cleaned.head(10))

Processing sentiment analysis...
   row_id sentiment_type  sentiment_score
0       1       compound           0.9381
0       2       compound           0.4118
0       3       compound          -0.3612
0       4       compound           0.9951
0       5       compound          -0.9895
0       6       compound           0.7962
0       7       compound          -0.9584
0       8       compound           0.9442
0       9       compound          -0.7436
0      10       compound          -0.5994


In [8]:
df_output = pd.merge(top_2021, t_df_cleaned, on='row_id', how='inner')
print(df_output.head(10))

   Unnamed: 0                                      all_awardings  \
0           0  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   
1           1  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   
2           2  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   
3           3  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   
4           4  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   
5           5  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   
6           6  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   
7           7  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   
8           8  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   
9           9  [{'award_sub_type': 'GLOBAL', 'award_type': 'g...   

   allow_live_comments               author  author_flair_css_class  \
0                 True                bjrob                     NaN   
1                False   Go9752788Stay37726                     NaN   
2                 True        manoffew

In [9]:
df_output[["sentiment_score"]].describe()

,sentiment_score
count,740.000000
mean,0.173345
std,0.701294
min,-0.996900
25%,-0.524600
50%,0.361200
75%,0.847475
max,0.998100


In [10]:
top_2021_final = df_output

In [11]:
top_2021_final['year'] = 2021

In [12]:
top_2021_final.head()

,Unnamed: 0,all_awardings,allow_live_comments,author,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_type,author_fullname,author_patreon_flair,...,author_flair_template_id,author_flair_text_color,removed_by_category,author_cakeday,date,combined,row_id,sentiment_type,sentiment_score,year
0,0,"[{'award_sub_type': 'GLOBAL', 'award_type': 'g...",True,bjrob,NaN,[],NaN,text,t2_1q50a99d,False,...,NaN,NaN,NaN,NaN,2021-03-03 17:43:17+00:00,CVS is vaccinating teachers regardless of stat...,1,compound,0.9381,2021
1,1,"[{'award_sub_type': 'GLOBAL', 'award_type': 'g...",False,Go9752788Stay37726,NaN,[],NaN,text,t2_5h8qz4ia,False,...,NaN,NaN,NaN,NaN,2021-02-24 21:45:44+00:00,I was observed today (online learning) and I h...,2,compound,0.4118,2021
2,2,"[{'award_sub_type': 'GLOBAL', 'award_type': 'g...",True,manoffewwords,NaN,[],NaN,text,t2_38lo5,False,...,NaN,NaN,NaN,NaN,2021-02-03 18:35:05+00:00,Has anyone else been unironically shown Sir Ke...,3,compound,-0.3612,2021
3,3,"[{'award_sub_type': 'GLOBAL', 'award_type': 'g...",True,SulkingDeath,NaN,[],NaN,text,t2_3z0ecem5,False,...,NaN,NaN,NaN,NaN,2021-12-07 22:08:11+00:00,I literally just stopped teaching in the middl...,4,compound,0.9951,2021
4,4,"[{'award_sub_type': 'GLOBAL', 'award_type': 'g...",True,textmasterj,NaN,[],NaN,text,t2_4hvmd,False,...,NaN,NaN,NaN,NaN,2021-02-25 12:15:16+00:00,Why do you want to send your kids back to us? ...,5,compound,-0.9895,2021


In [13]:
top_2021_final.to_csv('top_2021_final.csv')